In [ ]:
%pip install torch
%pip install numpy
%pip install pandas

In [ ]:
 
import numpy as np
import torch
import torchvision
#import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from torch.autograd import Variable
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn import preprocessing

In [ ]:
class my_model(nn.Module):
    def __init__(self, num_feature=380, num_class=1):
        super(my_model, self).__init__()
 
        self.layer_1 = nn.Linear(num_feature, 256)
        self.layer_2 = nn.Linear(256, 128)
        self.layer_3 = nn.Linear(128,64)
        self.layer_4 = nn.Linear(64, num_class)
        self.soft    = nn.Sigmoid()
        self.relu = nn.ReLU()
 
    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu(x)
        x = self.layer_2(x)
        x = self.relu(x)
        x = self.layer_3(x)
        x = self.relu(x)
        x = self.layer_4(x)
        x = self.soft(x)
        return x

In [ ]:
model=my_model(num_feature=380,num_class=1)
train = pd.read_csv('/content/drive/MyDrive/Project Files/train_set_123.csv')
train=np.transpose(train,axes=None)

#print("before",train.shape)
#print("after",train.shape)
#print(train.values.shape)
normalizer = preprocessing.Normalizer()

trainset_i = torch.tensor(normalizer.fit_transform(train.values[1:2000,1:381]),dtype=torch.float32)
trainset_l = torch.tensor(train.values[1:2000,-1],dtype=torch.float32)

dataset = TensorDataset( trainset_i,trainset_l)
trainloader = torch.utils.data.DataLoader(dataset,batch_size=32, shuffle=True)
dataiter = iter(trainloader)
images, labels = dataiter.next()




print(images.shape)
print(labels.shape)


torch.Size([32, 380])
torch.Size([32])


In [ ]:
criterion = nn.BCELoss()

optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
#optimizer=torch.optim.Adam(model.parameters(),lr=0.0001)
#torch.optim.Adadelta( model.parameters(),lr=0.07, rho=0.9, eps=1e-06, weight_decay=0)
#torch.optim.SparseAdam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)
time0 = time()
epochs = 1000
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(images)
        labels = labels.unsqueeze(1)
        #print(output.shape,labels.shape)
        loss = criterion(output, labels)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)   

Epoch 0 - Training loss: 0.6918885168575105
Epoch 1 - Training loss: 0.6919406652450562
Epoch 2 - Training loss: 0.6916973808455089
Epoch 3 - Training loss: 0.6917779123972333
Epoch 4 - Training loss: 0.6915625210792299
Epoch 5 - Training loss: 0.6917079639813256
Epoch 6 - Training loss: 0.6914355480481708
Epoch 7 - Training loss: 0.6914316964527917
Epoch 8 - Training loss: 0.6913163463274637
Epoch 9 - Training loss: 0.6914071677223085
Epoch 10 - Training loss: 0.691261405036563
Epoch 11 - Training loss: 0.6911617772919791
Epoch 12 - Training loss: 0.6911588708559672
Epoch 13 - Training loss: 0.6911789434296745
Epoch 14 - Training loss: 0.6907978814745707
Epoch 15 - Training loss: 0.6908728328962175
Epoch 16 - Training loss: 0.6908202634917365
Epoch 17 - Training loss: 0.6908282533524528
Epoch 18 - Training loss: 0.690524867602757
Epoch 19 - Training loss: 0.6905607713593377
Epoch 20 - Training loss: 0.6905491068249657
Epoch 21 - Training loss: 0.6905524560383388
Epoch 22 - Training lo

In [ ]:
testset = torch.tensor(normalizer.transform(train.values[2001:,1:381]),dtype=torch.float32)
testset_l = torch.tensor(train.values[2001:,-1],dtype=torch.float32)

In [ ]:
predictions = model(testset)
predictions=predictions.detach().numpy()


In [ ]:
from sklearn import metrics


In [ ]:
predictions

array([[8.91863783e-06],
       [9.99993205e-01],
       [5.88645662e-07],
       [1.00000000e+00],
       [3.68183896e-06],
       [5.05769222e-08],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.44368414e-06],
       [9.99999523e-01],
       [1.17993109e-01],
       [6.11217110e-04],
       [5.66032599e-04],
       [1.00000000e+00],
       [3.04351715e-06],
       [1.13560823e-08],
       [4.44131438e-05],
       [1.00000000e+00],
       [9.99999404e-01],
       [6.20004172e-08],
       [5.17329397e-07],
       [1.00000000e+00],
       [9.99992967e-01],
       [9.99963999e-01],
       [9.99999881e-01],
       [9.16761280e-08],
       [9.99996543e-01],
       [1.61359388e-08],
       [4.01358648e-06],
       [1.13776093e-06],
       [1.00000000e+00],
       [8.75851214e-01],
       [1.00000000e+00],
       [2.04660910e-08],
       [9.99999285e-01],
       [2.69217137e-09],
       [9.99998808e-01],
       [5.82616400e-09],
       [1.00000000e+00],
       [3.42954338e-01],


In [ ]:
accuracy=metrics.accuracy_score(testset_l,predictions.round())
print(accuracy)

0.9925
